# AI Feymann for UPINN

## Install module + dependencies

In [ ]:
!pip install numpy

In [ ]:
!pip install aifeynman

# Run DEMO

In [1]:
import aifeynman

In [ ]:
aifeynman.get_demos("example_data") # Download examples from server
aifeynman.run_aifeynman("./example_data/", "example1.txt", 60, "14ops.txt", polyfit_deg=3, NN_epochs=500)

# Generate Custom Function

In [9]:
import random

def getY(x0,x1):
  y = x0*x1*5.8
  return y

def getRow():
  [x0,x1]=[random.random() for _ in range(2)]
  y=getY(x0,x1)
  return str(x0) + " " + str(x1) + " " + str(y) + "\n"

with open("./example_data/my_example1.txt", "w") as f:
  for _ in range(1000):
    f.write(getRow())
f.close()

In [ ]:
aifeynman.run_aifeynman("./example_data/", "my_example1.txt", 30, "14ops.txt", polyfit_deg=3, NN_epochs=500)

# Ron on our models

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import aifeynman

In [3]:
"""
Lotka-Volterra and Apoptosis model: 
    one input for t followed by a scaling layer; 2 
    hidden layers of 64 units for the surrogate 
    solution; 2 hidden layers of 16 units for the hidden 
    component approximation; sigmoid activation
"""

class ScalingLayer(nn.Module):
    def __init__(self, scale_init_value=1e-2, bias_init_value=0):
        super().__init__()
        self.scale = nn.Parameter(torch.FloatTensor([scale_init_value]))
        self.bias = nn.Parameter(torch.FloatTensor([bias_init_value]))

    def forward(self, input):
        return input * self.scale + self.bias
    
    
class U(nn.Module):
    def __init__(self, in_shape, out_shape, hidden_shapes):
        super().__init__()
        
        self.layers = nn.ModuleList()
        self.layers.append(ScalingLayer())
        for hidden in hidden_shapes:
            self.layers.append(nn.Linear(in_shape, hidden))
            self.layers.append(nn.Sigmoid())
            in_shape = hidden
        self.layers.append(nn.Linear(hidden_shapes[-1], out_shape))
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
class F(nn.Module):
    def __init__(self, in_shape, out_shape, hidden_shapes):
        super().__init__()
        
        self.layers = nn.ModuleList()
        for hidden in hidden_shapes:
            self.layers.append(nn.Linear(in_shape, hidden))
            self.layers.append(nn.Sigmoid())
            in_shape = hidden
        self.layers.append(nn.Linear(hidden_shapes[-1], out_shape))
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [4]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
def create_data_file(u, f, filename_f1, filename_f2, n_samples=100):
    interval = [0, 3]
    t_samples = torch.linspace(interval[0], interval[1], n_samples).reshape(-1, 1)
    xy = u(t_samples)
    x, y = xy[:, 0], xy[:, 1]
    f12 = f(xy)
    f1, f2 = f12[:, 0], f12[:, 1]

    with open(filename_f1, "w") as f:
        for i in range(n_samples):
            f.write(f"{x[i]} {y[i]} {f1[i]}\n")

    with open(filename_f2, "w") as f:
        for i in range(n_samples):
            f.write(f"{x[i]} {y[i]} {f2[i]}\n")




In [9]:
def run(spacing, noise_level, f_n, brute_force_time=30, NN_epochs=500, n_samples=100):
    spacing_list = ["0.1", "0.2", "0.3", "0.4", "0.5", "0.6"]
    noise_level_list = ["0", "0.1", "0.01", "0.03", "0.05", "0.08", "0.008"]
    if spacing not in spacing_list or noise_level not in noise_level_list:
        raise ValueError("Spacing or noise level not trained")

    # load model
    upinn = U(1, 2, [64,64])
    fnn = F(2, 2, [16,16])
    upinn.load_state_dict(torch.load(f"../../upinn_models/LV/U_LV_exp2_sol_noise=0_space={spacing}.pt", map_location=DEVICE))
    fnn.load_state_dict(torch.load(f"../../upinn_models/LV/F_LV_exp2_sol_noise=0_space={spacing}.pt", map_location=DEVICE))
    upinn.eval()
    fnn.eval()

    data_dir = "./data"
    if not os.path.isdir(data_dir):
        os.mkdir(data_dir)

    f1_name = f"spacing={spacing}_noise={noise_level}_f1.txt"
    f2_name = f"spacing={spacing}_noise={noise_level}_f2.txt"

    # generate input files
    create_data_file(upinn, fnn, 
                     os.path.join(data_dir, f1_name), 
                     os.path.join(data_dir, f2_name), 
                    n_samples=n_samples)
    
    # once the data files are generated, move into another directory to run ai-feynman
    if not os.path.isdir("./solutions"):
        os.mkdir("./solutions")
    current_state_dir = "./solutions/current_state"
    if not os.path.isdir(current_state_dir):
        os.mkdir(current_state_dir)
    
    os.chdir(current_state_dir) # now we are in ./solutions/current_state


    # run ai feynman here
    if f_n == 1:
        aifeynman.run_aifeynman("../../data/", f1_name, brute_force_time, "7ops.txt", polyfit_deg=2, NN_epochs=NN_epochs, vars_name=["x", "y", "f1"])
        # clear useless files and save the solution
        os.system(f"cp ./results/solution_{f1_name} ../solution_{f1_name}")
    elif f_n == 2:
        aifeynman.run_aifeynman("../../data/", f2_name, brute_force_time, "7ops.txt", polyfit_deg=2, NN_epochs=NN_epochs, vars_name=["x", "y", "f2"])
        # clear useless files and save the solution
        os.system(f"cp ./results/solution_{f2_name} ../solution_{f2_name}")

    # move back to solutions folder
    os.chdir("../../") # now we are in ./

    # delete all content for current state of ai feynmann
    shutil.rmtree(current_state_dir)
    shutil.rmtree(data_dir)


In [10]:
# run("0.1", "0.01", 1)
# run("0.1", "0.03", 1)
# run("0.1", "0.05", 1)
# run("0.1", "0.1", 1)
# run("0.1", "0.08", 1)
# run("0.1", "0.008", 1)
# run("0.1", "0.01", 2)
# run("0.1", "0.03", 2)
# run("0.1", "0.05", 2)
# run("0.1", "0.1", 2)
# run("0.1", "0.08", 2)
# run("0.1", "0.008", 2)

run("0.1", "0", 1)
run("0.1", "0", 2)

run("0.2", "0", 1)
run("0.2", "0", 2)

run("0.3", "0", 1)
run("0.3", "0", 2)

run("0.4", "0", 1)
run("0.4", "0", 2)

run("0.5", "0", 1)
run("0.5", "0", 2)

run("0.6", "0", 1)
run("0.6", "0", 2)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve ../../data/spacing=0.1_noise=0_f1.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
          80  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......      80
Shuffling mystery data..
 Searching for best fit...
     25.316520751214      0.108840905130                      0               1             0.0000          2025.3217             0.3902             1.5830             3.6151            80.0000
     22.909655426655     -0.017151809287  

## Try to input the formula found by AI-feynman

In [ ]:
def func(x0, x1):
    return 0.029355282001*(x1+(np.exp(x1)+np.sqrt(np.pi)))

func = np.vectorize(func)

In [ ]:
# TODO order into folders for results
# get solution and save the best into another txt file 
# record the nn used and the input data